In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
from PIL import Image
import cv2

# EDA

The official source of this dataset was removed, there only it's copies on kaggle, huggingfacce or other data sources. So we don't provide any additional information about what it might tell us about the data, and in the code below we extract it from the files and create an EDA to better understand it.

p.s. to load dataset from kaggle you need to uncomments code block that do this for you

In [2]:
DATA_DIR = Path("./uavid_dataset")

## Dataload

In [3]:
# import kagglehub
# import shutil

# path = kagglehub.dataset_download("awsaf49/uavid-semantic-segmentation-dataset")

# shutil.move(path, DATA_DIR)

# print("Dataset moved to:", DATA_DIR.absolute())

## Extracting features

In this step, we iterate over each mask to get information about the class labels, how often we can see each class, or how much of the photo this class usually takes up. To get this information we should scrape it directly from train and valid data (for the valid data we do not have masks)

In [4]:
def get_label_metadata(img: np.ndarray) -> dict:
    img_flat = img.reshape(-1, 3)
    img_encoded = (img_flat[:, 0].astype(np.int32) << 16) | (img_flat[:, 1].astype(np.int32) << 8) | img_flat[:, 2].astype(np.int32)
    
    unique_encoded, counts = np.unique(img_encoded, return_counts=True)
    
    result = {}
    for encoded, count in zip(unique_encoded, counts):
        r = (encoded >> 16) & 0xFF
        g = (encoded >> 8) & 0xFF
        b = encoded & 0xFF
        result[f'#{r:02X}{g:02X}{b:02X}'] = int(count)
    
    return result

In [5]:
labels_list = list()

images_counter, labels_counter = 0, 0

for dirpath, dirnames, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        if filename.endswith(".png") and "Labels" in dirpath:
            filepath = f"{dirpath}/{filename}"

            img = cv2.imread(filepath, flags=cv2.IMREAD_COLOR_RGB)
            height, width, _ = img.shape

            colors_distribution = get_label_metadata(img)

            colors_distribution['filepath'] = filepath
            colors_distribution['height'] = height
            colors_distribution['width'] = width

            labels_list.append(colors_distribution)
            # unique_colors.update(colors_distribution.keys())

        if filename.endswith(".png") and "Images" in dirpath and ('train' in dirpath or 'valid' in dirpath):
            images_counter += 1
        
        if filename.endswith(".png") and "Labels" in dirpath and ('train' in dirpath or 'valid' in dirpath):
            labels_counter += 1

assert images_counter == labels_counter

After scrapping, we create a pandas dataframe, where we get a column with the path to the mask, the width and height of each image, as well as 8 unique columns with the names of the HEX colors that were extracted from each photo. In the corresponding column, we have the number of pixels of that color that the image has. 

In [6]:
df = pd.DataFrame(labels_list).fillna(0)
df = df[sorted(df.columns, reverse=True)]

In [7]:
df.shape

(270, 11)

Also, for better charts understanding, we created python dictionary with a mapping of HEX colors to what they mean on our masks

In [8]:
mapping = {
    '#C000C0': 'static car',        # (192, 0, 192)
    '#808000': 'low vegetation',   # (128, 128, 0)
    '#804080': 'road',            # (128, 64, 128)
    '#800000': 'building',        # (128, 0, 0)
    '#404000': 'human',          # (64, 64, 0)
    '#400080': 'moving car',     # (64, 0, 128)
    '#008000': 'tree',          # (0, 128, 0)
    '#000000': 'background clutter'  # (0, 0, 0)
}

categories = list(mapping.values())

df['area'] = df['width'] * df['height']
columns_share = list()

for column in mapping.keys():
    columns_share.append(f"{mapping.get(column)}".replace(" ", "_"))
    df[f"{mapping.get(column)}".replace(" ", "_")] = df[column] / df['area']

Here we see a bar chart where each bar corresponds to a class in the dataset, and its height shows how many photos that class is in.

In [9]:
presence_counts = {}

for cat in categories:
    col_name = f"{cat.replace(' ', '_')}"
    presence_counts[cat] = (df[col_name] > 0).sum()

presence_counts_sorted = dict(sorted(presence_counts.items(), key=lambda item: item[1], reverse=True))

fig = px.bar(x=list(presence_counts_sorted.keys()), 
             y=list(presence_counts_sorted.values()),
             labels={'x': 'Category', 'y': 'Number of Images'},
             title='Class Presence in Images',
             height=600)
fig.show()

Here we see a co-occurence matrix that captures frequency of how different pairs of classes appear together

In [10]:
co_occurrence = np.zeros((len(categories), len(categories)))

for _, row in df.iterrows():
    present = [i for i, cat in enumerate(categories) if row[f"{cat.replace(' ', '_')}"] > 0]
    for i in present:
        for j in present:
            co_occurrence[i, j] += 1

fig = px.imshow(co_occurrence, 
                x=categories, 
                y=categories,
                labels=dict(x="Category", y="Category", color="Count"),
                title="Co-occurrence Matrix",
                text_auto=True,
                height=600)
fig.show()

This plot answers for a question what part of the image a certain class occupies, as the result we see that on average the biggest class is the building and the smallest is human. This makes sense, because the photos are taken from the air and objects like building or green areas are larger than people or cars.

In [11]:
average_shares = pd.DataFrame(df[columns_share].mean()).rename(columns={0: 'mean_share'}).sort_values(by='mean_share', ascending=False)

fig = px.bar(average_shares, 
       labels={'index': 'Category', 'value': 'Average Share'},
       title='Average share by category on each images',
       height=600)

fig.show()

In [12]:
df['image_path'] = df['filepath'].str.replace('Labels', 'Images')
df['mask_path'] = df['filepath']

df_selected = df[['image_path', 'mask_path', 'width', 'height']]

Below you can see example of photo, mask and their blend

In [13]:
random_rows = df_selected.sample(n=3, random_state=42)

fig, axes = plt.subplots(3, 3, figsize=(15, 12))

for idx, (_, row) in enumerate(random_rows.iterrows()):
    
    original = Image.open(row['image_path'])
    mask = Image.open(row['mask_path'])
    blend = Image.blend(original.convert('RGB'), mask.convert('RGB'), alpha=0.5)
    
    axes[idx, 0].imshow(original)
    axes[idx, 0].set_title('Original')
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(mask)
    axes[idx, 1].set_title('Mask')
    axes[idx, 1].axis('off')
    
    axes[idx, 2].imshow(blend)
    axes[idx, 2].set_title('Blend')
    axes[idx, 2].axis('off')

    fig.text(0.5, 0.95 - idx * 0.31, row['image_path'], ha='center', fontsize=10)


plt.tight_layout()
plt.show()

In [14]:
df_selected['height'].value_counts(), df_selected['width'].value_counts()

(height
 2160    270
 Name: count, dtype: int64,
 width
 3840    170
 4096    100
 Name: count, dtype: int64)

In [ ]:
df_selected['data_part'] = df_selected['image_path'].apply(lambda x: x.split('/')[1]) 

/var/folders/_1/d9srt0k905g5yggt5g9bx29r0000gp/T/ipykernel_15008/4006987932.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Conclusions

We gained a better understanding of this dataset using different data visualizations. As a result of this EDA, we saved a .csv file with paths to images and masks. Additionally, the width and height of each photo were added to fix tensor shape mismatches in the following steps. 

In [16]:
# df_selected.to_csv('uavid_metadata.csv')